In [2]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
from keras import optimizers
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.models import load_model
import copy
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [3]:
import sys
#sys.path.insert(0, '../../')
sys.path.insert(0, '/home/luka/Developement/accetuation/')
from prepare_data import *

%run ../../prepare_data.py

In [25]:
data = Data('sl', accent_classification=True)
data.generate_data('syllables_accent_classification_train',
                   'syllables_accent_classification_test',
                   'syllables_accent_classification_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

LOADING DATA...
LOAD SUCCESSFUL!


In [4]:
data = Data('s', accent_classification=True)
data.generate_data('syllables_accent_classification_train',
                   'syllables_accent_classification_test',
                   'syllables_accent_classification_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

LOADING DATA...
LOAD SUCCESSFUL!


In [3]:
data = Data('l', accent_classification=True)
data.generate_data('letters_accent_classification_train',
                   'letters_accent_classification_test',
                   'letters_accent_classification_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

LOADING DATA...
LOAD SUCCESSFUL!


In [17]:
data = Data('l', save_generated_data=False, additional_letter_attributes=False, reverse_inputs=False, accent_classification=True)
data.generate_data('TEST_train',
                   'TEST_test',
                   'TEST_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [5]:
content = data._read_content('../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)

In [6]:
data.x_train = np.concatenate((data.x_train, data.x_test), axis=0)
data.x_other_features_train = np.concatenate((data.x_other_features_train, data.x_other_features_test), axis=0)
data.y_train = np.concatenate((data.y_train, data.y_test), axis=0)

In [29]:
num_examples = len(data.x_train) # training set size
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
actual_epoch = 5
num_fake_epoch = 20



# conv_input_shape=(23, 36)
conv_input_shape=(23, 30)
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

#x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x_conv)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [13]:
#model.load_weights('../word_accetuation/cnn_dictionary/test_accent_type_with_other_features/40_epoch.h5')
model.load_weights('/home/luka/Developement/accetuation/cnn/word_accetuation/cnn_dictionary/test_accent_type_with_other_features/40_epoch.h5')


In [27]:
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
# actual_epoch = 1
actual_epoch = 20
# num_fake_epoch = 2
num_fake_epoch = 20



# letters
# conv_input_shape=(23, 36)

# syllabled letters
conv_input_shape=(10, 252)


othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# letters
# x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
# x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)

# syllabled letters
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [127]:
model.load_weights('syllabled_letters/v1_0/20_epoch.h5')
#model = load_model('syllabled_letters/v2_5_3/40_epoch.h5', custom_objects={'actual_accuracy': actual_accuracy})

In [28]:
model.load_weights('syllabled_letters/v1_0/20_test_epoch.h5')

In [18]:
num_examples = len(data.x_train)  # training set size
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
# actual_epoch = 1
actual_epoch = 20
# num_fake_epoch = 2
num_fake_epoch = 20



# letters
# conv_input_shape=(23, 36)

# syllabled letters
conv_input_shape=(10, 5168)


othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# letters
# x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
# x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)

# syllabled letters
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [9]:
model.load_weights('syllables/v1_0/20_epoch.h5')

In [19]:
model.load_weights('syllables/v1_0/20_test_epoch.h5')

In [80]:
batch_size = 16
actual_epoch = 40
num_fake_epoch = 20

In [6]:
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
# actual_epoch = 1
actual_epoch = 20
# num_fake_epoch = 2
num_fake_epoch = 20



# letters
conv_input_shape=(23, 36)

# syllabled letters
# conv_input_shape=(10, 5168)


# othr_input = (140, )
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# letters
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)

# syllabled letters
# x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [6]:
model.load_weights('letters/v2_1/20_epoch.h5')
# model = load_model('letters/v2_1/40_epoch.h5', custom_objects={'actual_accuracy': actual_accuracy})

In [7]:
model.load_weights('letters/v2_1/20_test_epoch.h5')

In [34]:
y_array = np.asarray(data.y_test)
accentuation_length = (y_array > 0).sum()
scores = model.evaluate_generator(data.generator('test', batch_size, content_name='SlovarIJS_BESEDE_utf8.lex', content_location='../../data/'), 
                            accentuation_length/(batch_size))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


actual_accuracy: 77.70%


In [31]:
y_array = np.asarray(data.y_test)
accentuation_length = (y_array > 0).sum()
predictions = model.predict_generator(data.generator('test', batch_size, content_name='SlovarIJS_BESEDE_utf8.lex', content_location='../../data/'), 
                                      accentuation_length/(batch_size))

In [ ]:
accuracy, real_accuracy, errors = data.test_type_accuracy(predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print(accuracy)
print(real_accuracy)

In [9]:
accented_vowels = ['ŕ', 'á', 'ä', 'é', 'ë', 'ě', 'í', 'î', 'ó', 'ô', 'ö', 'ú', 'ü']
vowels = ['a', 'e', 'i', 'o', 'u', 'ŕ', 'á', 'ä', 'é', 'ë', 'ě', 'í', 'î', 'ó', 'ô', 'ö', 'ú', 'ü']

In [29]:
# PREDICT TEST DATA!!!
y_array = np.asarray(data.y_validate)
accentuation_length = (y_array > 0).sum()
predictions = model.predict_generator(data.generator('validate', batch_size, content_name='SlovarIJS_BESEDE_utf8.lex', content_location='../../data/'), 
                                      accentuation_length/(batch_size))

In [30]:
accuracy, real_accuracy, errors = data.test_type_accuracy(predictions, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
# print(accuracy)
print(real_accuracy)

95.98318025893549


In [154]:
#95.79732341465194
#95.57224634062008
#92.9019436239653
#92.52180260478697
#93.40152426795026
#93.04813861462216
print(accuracy)
print(real_accuracy)

96.44276702038435
96.25225709785241


In [24]:
errors[20:]

[[13352, 'besu', 'Ncmsd-', 'bésu', 'běsu'],
 [48252, 'besu', 'Ncmsl-', 'bésu', 'běsu'],
 [248, 'blejska', 'Agpnpa-', 'blëjska', 'bléjska'],
 [9642, 'blejska', 'Agpfsn-', 'blëjska', 'bléjska'],
 [9917, 'blejska', 'Agpnpn-', 'blëjska', 'bléjska'],
 [24836, 'blejska', 'Agpmdn-', 'blëjska', 'bléjska'],
 [49784, 'blejska', 'Agpmda-', 'blëjska', 'bléjska'],
 [18000, 'blejske', 'Agpfpn-', 'blëjske', 'bléjske'],
 [19866, 'blejske', 'Agpmpa-', 'blëjske', 'bléjske'],
 [36735, 'blejske', 'Agpfpa-', 'blëjske', 'bléjske'],
 [37288, 'blejske', 'Agpfsg-', 'blëjske', 'bléjske'],
 [8406, 'blejskega', 'Agpmsa-', 'blëjskega', 'bléjskega'],
 [46100, 'blejskega', 'Agpmsg-', 'blëjskega', 'bléjskega'],
 [46386, 'blejskega', 'Agpnsg-', 'blëjskega', 'bléjskega'],
 [11389, 'blejskem', 'Agpnsl-', 'blëjskem', 'bléjskem'],
 [23106, 'blejskem', 'Agpmsl-', 'blëjskem', 'bléjskem'],
 [9669, 'blejskemu', 'Agpmsd-', 'blëjskemu', 'bléjskemu'],
 [39372, 'blejskemu', 'Agpnsd-', 'blëjskemu', 'bléjskemu'],
 [189, 'blejski', 

In [30]:
errors = [[el[0], el[1][::-1], el[2], el[3][::-1], el[4][::-1]] for el in errors]

In [12]:
errors.sort(key=lambda x: x[1])

In [31]:
errors.sort(key=lambda x: x[1])
# name = 'letters_accent_classification'
# name = 'letters_accent_classification_test'
# name = 'syllables_accent_classification'
# name = 'syllables_accent_classification_test'
# name = 'syllabled_letters_accent_classification'
name = 'syllabled_letters_accent_classification_test'
output = open(name + '_errors.pkl', 'wb')
pickle.dump(errors, output)
output.close()
output = open(name + '_predictions.pkl', 'wb')
pickle.dump(predictions, output)
output.close()

In [8]:
pickle_input = open('letters_accent_classification_errors.pkl', 'rb')
letters_accent_classification_errors = pickle.load(pickle_input)
pickle_input = open('syllables_accent_classification_errors.pkl', 'rb')
syllables_accent_classification_errors = pickle.load(pickle_input)
pickle_input = open('syllabled_letters_accent_classification_errors.pkl', 'rb')
syllabled_letters_accent_classification_errors = pickle.load(pickle_input)

In [9]:
pickle_input = open('letters_accent_classification_predictions.pkl', 'rb')
letters_accent_classification_predictions = pickle.load(pickle_input)
pickle_input = open('syllables_accent_classification_predictions.pkl', 'rb')
syllables_accent_classification_predictions = pickle.load(pickle_input)
pickle_input = open('syllabled_letters_accent_classification_predictions.pkl', 'rb')
syllabled_letters_accent_classification_predictions = pickle.load(pickle_input)

In [33]:
accuracy, real_accuracy, errors = data.test_type_accuracy(letters_accent_classification_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print("letters_accent_classification_predictions")
print(accuracy)
print(real_accuracy)
accuracy, real_accuracy, errors = data.test_type_accuracy(syllables_accent_classification_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print("syllables_accent_classification_predictions")
print(accuracy)
print(real_accuracy)
accuracy, real_accuracy, errors = data.test_type_accuracy(syllabled_letters_accent_classification_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print("syllabled_letters_accent_classification_predictions")
print(accuracy)
print(real_accuracy)

letters_accent_classification_predictions
59.59504174738972
63.24868415997541
syllables_accent_classification_predictions
60.02880312719666
63.97671827576933
syllabled_letters_accent_classification_predictions
66.46321600630925
70.74801183295556


In [7]:
# TEST DATA!!!
pickle_input = open('letters_accent_classification_test_predictions.pkl', 'rb')
letters_accent_classification_predictions = pickle.load(pickle_input)
pickle_input = open('syllables_accent_classification_test_predictions.pkl', 'rb')
syllables_accent_classification_predictions = pickle.load(pickle_input)
pickle_input = open('syllabled_letters_accent_classification_test_predictions.pkl', 'rb')
syllabled_letters_accent_classification_predictions = pickle.load(pickle_input)

In [34]:
accuracy, real_accuracy, errors = data.test_type_accuracy(letters_accent_classification_predictions, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print("letters_accent_classification_predictions")
print(accuracy)
print(real_accuracy)
accuracy, real_accuracy, errors = data.test_type_accuracy(syllables_accent_classification_predictions, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print("syllables_accent_classification_predictions")
print(accuracy)
print(real_accuracy)
accuracy, real_accuracy, errors = data.test_type_accuracy(syllabled_letters_accent_classification_predictions, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print("syllabled_letters_accent_classification_predictions")
print(accuracy)
print(real_accuracy)

letters_accent_classification_predictions
96.25902240814716
96.06248386263879
syllables_accent_classification_predictions
94.25823375109297
93.99690162664601
syllabled_letters_accent_classification_predictions
96.22987638657912
95.98318025893549


In [14]:
def compare_outputs(outputs1, outputs2):
    outputs2_indices = [el[0] for el in outputs2]
    return [el for el in outputs1 if el[0] in outputs2_indices]

def get_unique_errors(outputs1, outputs2):
    outputs2_indices = [el[0] for el in outputs2]
    return [el for el in outputs1 if el[0] not in outputs2_indices]


#print(len(data.x_test))
#print(len(predictions))
print(len(letters_accent_classification_errors))
print(len(syllables_accent_classification_errors))
print(len(syllabled_letters_accent_classification_errors))
print(len(compare_outputs(letters_accent_classification_errors, syllables_accent_classification_errors)))
print(len(compare_outputs(letters_accent_classification_errors, syllabled_letters_accent_classification_errors)))
print(len(compare_outputs(syllables_accent_classification_errors, syllabled_letters_accent_classification_errors)))


print(1  - 7073.0/52058)
collective_errors = compare_outputs(letters_accent_classification_errors, compare_outputs(syllabled_letters_accent_classification_errors, syllables_accent_classification_errors))
print(len(collective_errors))

# print(letters_word_accetuation[:20])
# print(syllables_word_accetuation[:20])
# print(syllabled_letters_word_accetuation[:20])
print(collective_errors)

#unique_err = get_unique_errors(get_unique_errors(letters_accent_classification_errors, syllables_accent_classification_errors), syllabled_letters_accent_classification_errors)
#unique_err = get_unique_errors(get_unique_errors(syllables_accent_classification_errors, letters_accent_classification_errors), syllabled_letters_accent_classification_errors)
unique_err = get_unique_errors(get_unique_errors(syllabled_letters_accent_classification_errors, letters_accent_classification_errors), syllables_accent_classification_errors)

1951
2305
1871
875
1011
890
0.8641323139575089
599
[[19524, 'as', 'Ncmsn-', 'äs', 'ás'], [15290, 'bedast', 'Agpmsnn', 'bédast', 'bědast'], [43337, 'bedast', 'Agpmsan', 'bédast', 'bědast'], [257, 'bedasta', 'Agpmda-', 'bédasta', 'bědasta'], [3932, 'bedasta', 'Agpmdn-', 'bédasta', 'bědasta'], [23282, 'bedaste', 'Agpfpa-', 'bédaste', 'bědaste'], [29181, 'bedaste', 'Agpfsg-', 'bédaste', 'bědaste'], [38586, 'bedaste', 'Agpmpa-', 'bédaste', 'bědaste'], [46960, 'bedaste', 'Agpfpn-', 'bédaste', 'bědaste'], [4402, 'bedastem', 'Agpnsl-', 'bédastem', 'bědastem'], [21354, 'bedastem', 'Agpmsl-', 'bédastem', 'bědastem'], [21594, 'bedastih', 'Agpndg-', 'bédastih', 'bědastih'], [47357, 'bedastih', 'Agpmdg-', 'bédastih', 'bědastih'], [26461, 'bedastim', 'Agpmsi-', 'bédastim', 'bědastim'], [27117, 'bedastim', 'Agpnsi-', 'bédastim', 'bědastim'], [31494, 'bedastim', 'Agpmpd-', 'bédastim', 'bědastim'], [37396, 'bedastim', 'Agpnpd-', 'bédastim', 'bědastim'], [39351, 'bedastim', 'Agpfpd-', 'bédastim', 'bědas

In [12]:
ensemble_predictions = np.mean( np.array([ letters_accent_classification_predictions, syllables_accent_classification_predictions, syllabled_letters_accent_classification_predictions ]), axis=0 )
accuracy, real_accuracy, errors = data.test_type_accuracy(ensemble_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print(accuracy)
print(real_accuracy)

97.29059548554133
97.14549156709825


In [ ]:
output = open('ensemble_errors.pkl', 'wb')
pickle.dump(errors, output)
output.close()
output = open('ensemble_predictions.pkl', 'wb')
pickle.dump(predictions, output)
output.close()

In [15]:
%run ../../prepare_data.py
test = Data('s', accent_classification=True)

In [16]:
ensemble_predictions = np.mean( np.array([ letters_accent_classification_predictions, syllables_accent_classification_predictions, syllabled_letters_accent_classification_predictions ]), axis=0 )
accuracy, real_accuracy, errors = test.test_type_accuracy(ensemble_predictions, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
# accuracy, real_accuracy, errors = data.test_type_accuracy(ensemble_predictions, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print(accuracy)
print(real_accuracy)

58327
54222
52255
1967
54222
96.54705368011383
96.3723211980377


In [11]:
1 - len(errors)/float(len(ensemble_predictions))

0.9662763385739023

In [9]:
output = open('ensemble_test_errors.pkl', 'wb')
pickle.dump(errors, output)
output.close()
output = open('ensemble_test_predictions.pkl', 'wb')
pickle.dump(ensemble_predictions, output)
output.close()

In [15]:
print(len(unique_err))
#collective_errors[:1000]
unique_err[:1000]

569


[[14677, 'alergičnih', 'Agpndg-', 'alérgičnih', 'alěrgičnih'],
 [23706, 'anket', 'Ncfpg-', 'ankět', 'ankét'],
 [49392, 'anket', 'Ncfdg-', 'ankět', 'ankét'],
 [32228, 'bežne', 'Agpmpa-', 'běžne', 'béžne'],
 [43777, 'bežne', 'Agpfpa-', 'běžne', 'béžne'],
 [47661, 'bežne', 'Agpfpn-', 'běžne', 'béžne'],
 [10195, 'bežnem', 'Agpnsl-', 'běžnem', 'béžnem'],
 [46867, 'bežnem', 'Agpmsl-', 'běžnem', 'béžnem'],
 [14835, 'bežni', 'Agpfdn-', 'běžni', 'béžni'],
 [17234, 'bežni', 'Agpndn-', 'běžni', 'béžni'],
 [23480, 'bežni', 'Agpmpn-', 'běžni', 'béžni'],
 [48038, 'bežni', 'Agpfda-', 'běžni', 'béžni'],
 [3218, 'bežnih', 'Agpfdg-', 'běžnih', 'béžnih'],
 [7755, 'bežnih', 'Agpfdl-', 'běžnih', 'béžnih'],
 [16357, 'bežnih', 'Agpndl-', 'běžnih', 'béžnih'],
 [17446, 'bežnih', 'Agpndg-', 'běžnih', 'béžnih'],
 [20978, 'bežnih', 'Agpmdg-', 'běžnih', 'béžnih'],
 [21527, 'bežnih', 'Agpfpg-', 'běžnih', 'béžnih'],
 [23626, 'bežnih', 'Agpmpl-', 'běžnih', 'béžnih'],
 [31598, 'bežnih', 'Agpfpl-', 'běžnih', 'béžnih'],